In [ ]:
import pandas as pd

In [ ]:
colunas_nomes = [
    'ID', 'DATA', 'HORA_MINUTO', 'MUNICIPIO', 'BAIRRO',
    'ENDERECO', 'ORIGEM_CHAMADO', 'TIPO', 'SUBTIPO',
    'SEXO', 'IDADE', 'MOTIVO_FINALIZACAO', 'MOTIVO_DESFECHO'
]


df_2025 = pd.read_csv('../data/samu_2025.csv', header=None, names=colunas_nomes)

df_2024 = pd.read_csv('../data/samu_2024.csv', header=0, names=colunas_nomes)
df_2023 = pd.read_csv('../data/samu_2023.csv', header=0, names=colunas_nomes)

df_unificado = pd.concat([df_2025, df_2024, df_2023], ignore_index=True)

df_unificado['ID'] = df_unificado.index
df_unificado.set_index('ID', inplace=True)

df_unificado['DATA'] = df_unificado['DATA'].astype(str).str.split('T').str[0]
df_unificado['DATA'] = pd.to_datetime(df_unificado['DATA'], errors='coerce').dt.date

df_unificado['HORA_MINUTO'] = df_unificado['HORA_MINUTO'].astype(str).str.strip().str[:8]
df_unificado['HORA_MINUTO'] = pd.to_datetime(df_unificado['HORA_MINUTO'], format='%H:%M:%S', errors='coerce').dt.time
df_unificado['IDADE'] = pd.to_numeric(df_unificado['IDADE'], errors='coerce')


print(df_unificado[['DATA', 'HORA_MINUTO', 'IDADE']].info())
df_unificado.head()

In [ ]:
print("Tipo real na coluna DATA:", type(df_unificado['DATA'].iloc[0]))

print("Tipo real na coluna HORA:", type(df_unificado['HORA_MINUTO'].iloc[0]))

In [ ]:
mediana_idade = df_unificado['IDADE'].median()

df_unificado['IDADE'].fillna(mediana_idade, inplace=True)
df_unificado['IDADE'] = df_unificado['IDADE'].astype(int)

qtd_nulos = df_unificado['IDADE'].isnull().sum()

print("Quantidade de valores nulos na coluna IDADE após preenchimento:", qtd_nulos)

In [ ]:
df_unificado['MOTIVO_FINALIZACAO'] = df_unificado['MOTIVO_FINALIZACAO'].fillna('SEM FINALIZAÇÃO')

print("Quantidade de nulos após tratamento:", df_unificado['MOTIVO_FINALIZACAO'].isnull().sum())

df_unificado[['MOTIVO_FINALIZACAO']].info()

In [ ]:
df_unificado['ENDERECO'] = df_unificado['ENDERECO'].fillna('NÃO INFORMADO')

df_unificado['ORIGEM_CHAMADO'] = df_unificado['ORIGEM_CHAMADO'].fillna('NÃO INFORMADO')

print(df_unificado[['ENDERECO', 'ORIGEM_CHAMADO']].isnull().sum())

df_unificado.head()

In [ ]:
colunas_restantes = ['SEXO', 'SUBTIPO', 'TIPO', 'MUNICIPIO', 'BAIRRO']

df_unificado[colunas_restantes] = df_unificado[colunas_restantes].fillna('NÃO INFORMADO')

print("Contagem Final de Nulos:")
print(df_unificado.isnull().sum())

df_unificado.info()

In [ ]:
colunas_texto = ['MUNICIPIO', 'BAIRRO', 'ENDERECO', 'ORIGEM_CHAMADO', 'TIPO', 'SUBTIPO', 'SEXO', 'MOTIVO_FINALIZACAO', 'MOTIVO_DESFECHO']

for col in colunas_texto:
    df_unificado[col] = df_unificado[col].astype(str).str.upper().str.strip()

In [ ]:
colunas_texto = ['MUNICIPIO', 'BAIRRO', 'ENDERECO', 'ORIGEM_CHAMADO', 'TIPO', 'SUBTIPO', 'SEXO', 'MOTIVO_FINALIZACAO', 'MOTIVO_DESFECHO']

for col in colunas_texto:
    print(f"\nValores Únicos em {col}")
    valores = sorted(df_unificado[col].unique())
    print(valores)

In [ ]:
valores_para_limpar = [
    '93999830', 'ANI/ALI','JOSELENE', 'JUSELITA', 
    'MARCILIA', 'R MA','RAYSSA', 'R  CELIA','JAGUARIB'
    'MONICA', 'AV NORTE', '00', 'MONIQUE', 'CARLOS', 'SANDRO',
    'EDVALDO', 'RECIFE', 'EDIMILSO', 'MARIA', 'MANOEL R', 'TEC ENF',
    'ANTONIO'
]

df_unificado['ORIGEM_CHAMADO'] = df_unificado['ORIGEM_CHAMADO'].replace(valores_para_limpar, 'NÃO INFORMADO')

df_unificado['ORIGEM_CHAMADO'] = df_unificado['ORIGEM_CHAMADO'].replace('ESTAB PR', 'ESTABELECIMENTO PRIVADO')
df_unificado['ORIGEM_CHAMADO'] = df_unificado['ORIGEM_CHAMADO'].replace('ESTAB PU', 'ESTABELECIMENTO PUBLICO')

In [ ]:
nulos_finalizacao = df_unificado['MOTIVO_FINALIZACAO'].isnull().sum()
print("Quantidade de valores nulos na coluna MOTIVO_FINALIZACAO após todas as correções:", nulos_finalizacao)

In [ ]:
colunas_checagem = ['DATA', 'HORA_MINUTO', 'MUNICIPIO', 'BAIRRO', 
    'ENDERECO', 'ORIGEM_CHAMADO', 'TIPO', 'SUBTIPO', 
    'SEXO', 'IDADE', 'MOTIVO_FINALIZACAO', 'MOTIVO_DESFECHO']


qtd_antes = len(df_unificado)
df_unificado.drop_duplicates(subset=colunas_checagem, keep='first', inplace=True)
qtd_depois = len(df_unificado)

print(f"Linhas antes: {qtd_antes}")
print(f"Linhas depois: {qtd_depois}")

In [ ]:
mapa_dias = {
    'Monday': 'SEGUNDA-FEIRA', 'Tuesday': 'TERCA-FEIRA', 'Wednesday': 'QUARTA-FEIRA',
    'Thursday': 'QUINTA-FEIRA', 'Friday': 'SEXTA-FEIRA', 'Saturday': 'SABADO', 'Sunday': 'DOMINGO'
}

df_unificado['DATA'] = pd.to_datetime(df_unificado['DATA'])
df_unificado['DIA_SEMANA'] = df_unificado['DATA'].dt.day_name().map(mapa_dias)


def definir_turno(hora_minuto):
    try:
        hora = int(str(hora_minuto)[:2])
        
        if 6 <= hora < 12:
            return 'MANHA'
        elif 12 <= hora < 18:
            return 'TARDE'
        elif 18 <= hora <= 23:
            return 'NOITE'
        else:
            return 'MADRUGADA'
    except:
        return 'NAO INFORMADO'

df_unificado['TURNO'] = df_unificado['HORA_MINUTO'].apply(definir_turno)

print("Novas colunas geradas:")
df_unificado[['DATA', 'DIA_SEMANA', 'HORA_MINUTO', 'TURNO']].head()

In [ ]:
df_unificado['ANO_ORIGEM'] = df_unificado['DATA'].dt.year.astype('Int64')

In [ ]:
df_unificado.head(10)